In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Tesis/tesis/classificador')
else:
    os.chdir('../classificador')

In [4]:
from datasets.speech_commands_dataset import *
from transforms import transforms_wav as twav
from transforms import transforms_stft as tstft
from torchvision.transforms import *

In [ ]:
background_noise = 'datasets/_background_noise_'

In [ ]:
data_aug_transform = Compose([
    twav.ChangeAmplitude(), 
    twav.ChangeSpeedAndPitchAudio(), 
    twav.FixAudioLength(), 
    tstft.ToSTFT(), 
    tstft.StretchAudioOnSTFT(), 
    tstft.TimeshiftAudioOnSTFT(), 
    tstft.FixSTFTDimension()])
bg_dataset = BackgroundNoiseDataset(background_noise, data_aug_transform)
add_bg_noise = tstft.AddBackgroundNoiseOnSTFT(bg_dataset)
train_feature_transform = Compose([
    tstft.ToMelSpectrogramFromSTFT(n_mels=n_mels), 
    tstft.DeleteSTFT(), 
    twav.ToTensor('mel_spectrogram', 'input')])

In [ ]:

train_dataset = SpeechCommandsDataset(args.train_dataset,
                                Compose([twav.LoadAudio(),
                                         data_aug_transform,
                                         add_bg_noise,
                                         train_feature_transform]))